In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# Klassifikation mit *k*-Nearest-Neighbors
Ziel dieser Übung ist das eigenständige Implementieren des Klassifikations-Algorithmus *k*-Nearest-Neighbors innerhalb des Jupyter-Notebooks. Implementieren sie folgende Variante des *k*-Nearest-Neighbors:
- Alle Attribute sind vor der Benutzung auf den Wertebereich $[0;1]$ zu normieren. Beachten Sie dabei, dass für das "Training" des Klassifikators keine Informationen aus den Testdaten verwendet werden dürfen.
- Als Distanzfunktion nutzen Sie bitte die euklidische Distanz.
- Für das Abstimmen der *k* nächsten Nachbarn soll es vier Varianten geben, die mittels eines Parameters an die Klasse übergeben werden können:
    1. Eine einfache Mehrheitsabstimmung unter den Nachbarn
    2. Jeder Nachbar wird mit dem inversen Quadrat der Distanz gewichtet.
    3. Die Stimmen einer Klasse werden mit dem Inversen ihrer Durchschnittsdistanz gewichtet.
    4. Eine Mehrheitsabstimmung gewichtet nach der Verteilung der Klassen.
   
Sie dürfen die Pakete **collections**, **math** und **numpy** für Ihre Implementierung nutzen. Für die Ausführung der Tests benötigen Sie außerdem **pandas**.

# Aufgaben für 6ECTS
Der *k*-nearest Neighbor Algorithmus verwendet für die Klassifikation verschiedene Parameter: Es muss ein fester Wert für **k** und eine der Entscheidungsstrategien gewählt werden. Doch wie wählt man die Parameter sinnvoll? Eine Möglichkeit liefert die Kreuzvalidierung, welche Sie implementieren sollen:
Die Funktion *train* hat einen Parameter **ks**. Falls dieser nicht *None* ist, soll dieser Parameter genutzt werden, um eine Liste von Möglichen *k* Werten zu übergeben. Sie sollen dann wie folgt vorgehen, um aus diser Liste von *k* Werten das bestmögliche *k* und die bestmögliche der 4 Stragtegien zu wählen:
- Teilen Sie die Daten zufällig in 5 gleichgroße Teile auf. Nutzen Sie dafür die vorgegbene Funktion *split*.
- Gehen Sie für jedes Kombination für *k* und jeder Stratege *s* wie folgt vor um die besten Werte zu finden:
- Für jede der 5 Datensatzteile "trainieren" sie auf den üblichen 4 Teilen und klassifizieren die Daten des nicht zum Training verwerndeten Teil. Zählen Sie die richtigen Klassifikationen.
- Summieren Sie auf, sodass sie die richtigen Entscheidungen über alle 5 Teile erhalten.
- Nun haben sie für jedes Paar (*k*,*s*) eine Anzahl an richtigen Entscheidungen über die 5 Teile.
- Wählen Sie nun die beste Kombination aus und "trainieren" sie auf den ganzen Datensatz. Speichern Sie außerdem das "beste" Paar (*k*,*s*).
- Wird nun die Funktion *predict* mit *best_combination=True* aufgerufen, so sollen der ermittelte Wert für *k* und die ermittelte Strategie statt die übergebenen Werte genutzt werden.


Damit die Aufgabe als sinnvoll bearbeitet gilt, sind folgende Anforderungen zu erfüllen:
- Bei einem Durchlauf durch den Iris-Datensatz sollen keine Ausführungsfehler bestehen und (sehr) gute Werte für die Accuracy geliefert werden.
- Abgabe der Übung bis 06.02.2023 23:59 Uhr im Moodle-Kurs. 

In [29]:
import math
from collections import Counter, defaultdict
import numpy as np
import random # Just needed for 6ECTS


def euclidean_dist(x, y):
    # TODO
    #prüfen ob die beiden Vektoren x und y die gleiche Dimension 
    assert len(x) == len(y)
    #die Summe der quadrierten Differenzen berechnet 
    squared_dist = sum((xi - yi) ** 2 for xi, yi in zip(x, y))
    return math.sqrt(squared_dist)

def split(X, Y, n=5):
    "Split the training points and labels into 5 equal sized parts. Just needed for 6 ECTS."
    m = list(range(len(X)))
    X = np.array(X)
    Y = np.array(Y)
    indices = [np.array(m[i::n]) for i in range(n)]
    X = [X[i].tolist() for i in indices]
    Y = [Y[i].tolist() for i in indices]
    return X, Y

class KNN():
    def __init__(self, dist_fun=euclidean_dist):
        self.dist_fun = dist_fun
        self.strategies = ['majority', 'inverse_squared_distance', 'inverse_avg_distance', 'distribution']
        self.k = None
        self.strategy = None
        self.trainingDataWert = None
        self.trainingDataLabel = None
        self.dataMin = None
        self.dataMax = None
        
    
    def train(self, X, Y, ks=None):
        """ Train this classifier. Takes a list of samples X and a list of class-labels Y.
        Each sample is a list of numeric values. Each label is a string.
        The parameter ks ist just needed for 6ECTS."""
        #x die numerische Daten,y labels/Bezeichnung, was macht gibt zurück keine Ahnung
        assert(len(X) == len(Y))
        
        # TODO
        #auf den Wertebereich [0;1] zu normieren
        data = np.array(X)
        self.dataMin = np.min(data)
        self.dataMax = np.max(data)
        normalizedData = (data-np.min(data))/(np.max(data)-np.min(data))
        if ks == None:
            #dictionary erstellen mit label und nomierte Werte
            self.trainingDataWert = normalizedData
            self.trainingDataLabel = Y
        else:
            return None
    
    def predict(self, X, k=3, strategy='majority', best_combination=False):
        """ Takes a list of samples X. Returns a list of predicted labels for the samples.
        The parameter best_combination ist just needed for 6ECTS."""
        return [self.predict_sample(x, k, strategy) for x in X]
    
    def predict_sample(self, x, k=3, strategy='majority'):
        """ Predicts the label for a single sample x. """
        
        # TODO
        #x normalisieren
        xNormilized = (x-self.dataMin)/(self.dataMax-self.dataMin)
        # für alle traing Data,  Abstand zwischen x und data berechnen, speichert in eine List für folgenden Fällen
        distances = [(self.trainingDataLabel[i], self.dist_fun(self.trainingDataWert[i], xNormilized)) for i in range(len(self.trainingDataWert))]
        distances.sort(key=lambda x:x[1])
        #         print('distances')
#         print(distances)
        #Eine einfache Mehrheitsabstimmung unter den Nachbarn
        if strategy == 'majority':
            #Sortieren in aufsteigender Reihenfolge, basierend auf einem Abstandswert
            #Ermitteln der ersten k Stichproben
            neighbors = [distances[i][0] for i in range(k)]
            prediction = max(neighbors,key=neighbors.count)
            
        # Jeder Nachbar wird mit dem inversen Quadrat der Distanz gewichtet.
        if strategy == 'inverse_squared_distance':
            neighbors = [(distances[i][0],distances[i][1]) for i in range(k)]
            neighbor_counts = {}
            for (neighbor,distance) in neighbors:
                # die dict.get()-Methode verwendet, um die aktuelle Anzahl der einzelnen Nachbarn abzurufen, 
                #und standardmäßig 0, wenn der Nachbar nicht im Wörterbuch enthalten ist
                # jede Klase in dictionary mit 
                neighbor_counts[neighbor] = neighbor_counts.get(neighbor, 0) + 1/distance
            prediction = max(neighbor_counts, key=neighbor_counts.get)

            
            
#             print(prediction)
        # Die Stimmen einer Klasse werden mit dem Inversen ihrer Durchschnittsdistanz gewichtet.
#         if strategy == 'inverse_avg_distance':
#             #for all class in traingset
#                 X_class = X[y == class_label]

#                 # Calculate centroid of the subset
#                 centroid = np.mean(X_class, axis=0)

#                 # Calculate distances between each sample and the centroid
#                 dists = np.sqrt(np.sum((X_class - centroid)**2, axis=1))

#                 # Calculate average distance of the subset
#                 avg_dist = np.mean(dists)

#             # Calculate inverse of average distance
#             inv_avg_dist = 1 / avg_dist
        
#         # Eine Mehrheitsabstimmung gewichtet nach der Verteilung der Klassen.
#         if strategy == 'distribution':
        
        return prediction


# Evaluierung des Klassifikators
Mit diesem Code können Sie Ihre Implementierung anhand des mitgelieferten IRIS-Datensatzes testen. Probieren Sie auch verschiedene (sinnvolle) Werte für den Parameter *k*. Bitte ansonsten in diesem Teil nichts mehr ändern.

In [30]:
def accuracy(predictions, targets):
    """ Calculates the accuracy for the given class predictions and true classes."""
    assert(len(predictions) == len(targets))
    n_correct = len([p for p,t in zip(predictions, targets) if p==t])
    return n_correct/len(predictions)

In [31]:
def confusion_matrix(predictions, targets):
    """ Returns a tuple (labels, m) where m is the confusion matrix and 
    labels is the list of matrix rows/columns in same order as in the matrix.
    Rows in the confusion matrix indicate the true target label
    whereas the columns indicate the predicted label of samples. """
    assert(len(predictions) == len(targets))
    
    # Map each label to an index.
    #Return a set that contains all items from both sets, duplicates are excluded:
    unique_vals = list(set(predictions).union(targets))
    mapping = {label: index for index, label in enumerate(unique_vals)}
    
    # Build and fill the confusion matrix.
    #??
    m = [[0]*len(mapping) for _ in range(len(mapping))]
    for p, t in zip(predictions, targets):
        row, col = mapping[t], mapping[p]
        #row ist target, col ist prediction mit indexanzahl 
        m[row][col] += 1
    return unique_vals, m

In [32]:
import pandas as pd

# Load the csv and drop duplicate entries.
data = pd.read_csv('iris.csv').drop_duplicates()

# Draw a random sample without replacement for the test data.
test_data = data.sample(n=50)


In [33]:
# The other samples are used as training data.
train_data = data.loc[data.index.drop(test_data.index), :]

def df_to_vectors(df):
    """Takes a pandas data-frame from the iris dataset as input.
    Returns a tuple (X, Y) where Y is a list of class labels and X is the list of sample-vectors
    with each vector represented as a list of numeric values."""
    df = df.copy()
    classes = df['species']
    del df['species']
    return df.values.tolist(), classes.values.tolist()

# Convert train and test-data to lists of vectors and class labels.
X_train, Y_train = df_to_vectors(train_data)

# print('X_train:')
# print(X_train)
# print('Y_train:')
# print(Y_train)

In [34]:
X_test, Y_test = df_to_vectors(test_data)

In [35]:
clf = KNN()
clf.train(X_train, Y_train)
predictions = clf.predict(X_test, strategy='inverse_squared_distance', k=3)


In [36]:

for strategy in clf.strategies:
    predictions = clf.predict(X_test, strategy=strategy, k=3)
    print('Accuracy of strategy {}: {}'.format(strategy, accuracy(predictions, Y_test)))
    labels, matrix = confusion_matrix(predictions, Y_test)
    print('Confusion matrix:')
    print('\n'.join([str(row) for row in matrix]))
    print('----------')

Accuracy of strategy majority: 0.96
Confusion matrix:
[18, 2, 0]
[0, 16, 0]
[0, 0, 14]
----------
Accuracy of strategy inverse_squared_distance: 0.96
Confusion matrix:
[18, 2, 0]
[0, 16, 0]
[0, 0, 14]
----------


UnboundLocalError: local variable 'prediction' referenced before assignment

# Delete the following Lines if you just need 3 ECTS!

In [174]:
clf = KNN()
clf.train(X_train, Y_train, ks=[1, 3, 5, 7])
print("Best combination:")
print("k:", clf.k)
print("strategy:", clf.strategy)
print("-----------")
predictions = clf.predict(X_test, best_combination=True)
labels, matrix = confusion_matrix(predictions, Y_test)
print('Accuracy {}'.format(accuracy(predictions, Y_test)))
print('Confusion matrix:')
print('\n'.join([str(row) for row in matrix]))

Best combination:
k: None
strategy: None
-----------


TypeError: object of type 'NoneType' has no len()